Primer paso es leer BD del servidor donde se tenga la tabla de los pesometros antiguos con problemas.
En este caso es el servidor de google

In [42]:
import mysql.connector
import time
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter


db = mysql.connector.connect(host="35.198.8.176",user="alvarov",passwd="lata5comprar",db="bsa_loblanco",charset = "utf8" )        # el nombre de la base de datos
#analisisBDInaplast
cur = db.cursor()
print("conexion a gc")
consulta="SHOW TABLES"
cur.execute(consulta)
for row in cur.fetchall():
    print (str(row[0]))
print (" ")


conexion a gc
camiones_reg
cargador_reg
pesometroreparado2
pesometros
 


Los datos en bruto estan en la tabla pesometro columna acc1 pesometro integral y acc2 pesometro grava 3/4.
row[1]=fecha, row[5]=integral, row[6]=grava3/4

In [43]:
consulta="DESCRIBE pesometros"
cur.execute(consulta)
for row in cur.fetchall():
    print (str(row[0]))
print ("se termino la descripcion de pesometros ")


id
fecha
tph1
tph2
tph3
acc1
acc2
acc3
diff1
diff2
diff3
se termino la descripcion de pesometros 


vamos a leer el primer y ultimo registro de la tabla pesometro, en la consulta anterior se muestran los nombre de las columnas


In [60]:
db = mysql.connector.connect(host="35.198.8.176",user="alvarov",passwd="lata5comprar",db="bsa_loblanco",charset = "utf8" )        # el nombre de la base de datos
#analisisBDInaplast
cur = db.cursor()
print("conexion a gc")

integralx=[]
gravax=[]
fechaIni="'2019-08-07 08:00:03'"
fechaFin="'2019-08-16 22:47:00'" 

#consula
tabla="pesometros"
consulta="SELECT * FROM "+tabla+" WHERE fecha >= "+fechaIni+ "ORDER BY id ASC LIMIT 1"
print(consulta)
cur.execute(consulta)
print ("El primer registro de la consulta ")
for row in cur.fetchall():
     print (str(row[0]) + " " +str(row[1])+" "+str(row[2]) + " " +str(row[3]))
     integralx.append(row[5])
     gravax.append(row[6])
print (" ")
print ("El ultimo registro de la consulta ")
tabla="pesometros"
consulta="SELECT * FROM "+tabla+" WHERE fecha <= "+fechaFin+ "ORDER BY id DESC LIMIT 1"
cur.execute(consulta)
for row in cur.fetchall():
     print (str(row[0]) + " " +str(row[1])+" "+str(row[2]) + " " +str(row[3]))
     integralx.append(row[5])
     gravax.append(row[6])
print (" ")
print(integralx)
print(gravax)
restaint=integralx[0]
restagrav=gravax[0]



conexion a gc
SELECT * FROM pesometros WHERE fecha >= '2019-08-07 08:00:03'ORDER BY id ASC LIMIT 1
El primer registro de la consulta 
54159986 2019-08-07 19:20:13 -1 -1
 
El ultimo registro de la consulta 
54314731 2019-08-16 22:45:07 0 0
 
[254724709600, 265377798100]
[84099887700, 87408469800]


Ahora vamos a reparar los datos


In [57]:
db = mysql.connector.connect(host="35.198.8.176",user="alvarov",passwd="lata5comprar",db="bsa_loblanco",charset = "utf8" )        # el nombre de la base de datos
#analisisBDInaplast
cur = db.cursor()
print("conexion a gc")
fechaIni="'2019-08-07 23:00:03'"
fechaFin="'2019-08-07 23:59:00'" 
#consula
tabla="pesometros"

integral=[]
grava=[]
restaint=integralx[0]
restagrav=gravax[0]

anteriorint=0  # registro anterior de la base antigua
anteriorgrav=0
contador=0
print(integralx)
print(gravax)



conexion a gc
[254724709600, 265377798100]
[84099887700, 87408469800]


In [ ]:
#de atras deben venir los arreglos 
#restaint=integralx[0]
#restagrav=gravax[0]
#que son los valores acumulados, los cuales vamos a venecitar para restar
db = mysql.connector.connect(host="35.198.8.176",user="alvarov",passwd="lata5comprar",db="bsa_loblanco",charset = "utf8" )        # el nombre de la base de datos
#analisisBDInaplast
cur = db.cursor()
print("vamos a comenzar la revision e insercion")
consulta="select * FROM "+tabla+" WHERE fecha between "+fechaIni+" and "+fechaFin+" "  
print(consulta)
cur.execute(consulta)

anteriorint=0  # registro anterior de la base antigua
anteriorgrav=0
contador=0
print(integralx)
print(gravax)


for row in cur.fetchall():
     #print (str(row[0]) + " " +str(row[1])+" "+str(row[2]) + " " +str(row[3]))
    if contador==0:
         anteriorint=row[5]
         anteriorgrav=row[6]
         anteriorintsql=0
         anteriorgravasql=0
    if row[5]-anteriorint>3000:

        print("entro aca")
        restaint=restaint+row[5]-anteriorint
        print ("cambio de Integral")
    if row[6]-anteriorgrav>3000:
        restagrav=restagrav+row[6]-anteriorgrav
              
   
    print(str(contador)+"  "+str(row[1])+"   "+str(row[5])+"   "+str(row[5]-restaint)+"---------"+str(row[6])+"   "+str(row[6]-restagrav))
    integral.append(row[5]-restaint)
    grava.append(row[6]-restagrav)
    #aca debo hacer el insert a la nueva tabla
    consulta="INSERT INTO `pesometroreparado3` (`hora`, `acc1`, `acc2`, `acc3`)VALUES ('"+str(row[1])+"', '"+str(row[5]-restaint-anteriorintsql)+"', '"+str(row[6]-restagrav-anteriorgravasql)+"', NULL);"
    #print(consulta)
    #print(anteriorint)
    cur.execute(consulta)
    db.commit()
    anteriorint=row[5]
    anteriorgrav=row[6]
    anteriorintsql=row[5]-restaint
    anteriorgravasql=row[6]-restagrav
    contador=contador+1

print("la produccion de integral es : ")
print(str(integral[len(integral)-1]))
print("la produccion de integral es en toneladas: ")
print(str(integral[len(integral)-1]*100/1000))

print("la produccion de grava es : ")
print(str(grava[len(grava)-1]))

print("la produccion de grava en toneladas es : ")
print(str(grava[len(grava)-1]*100/1000))

vamos a comenzar la revision e insercion
select * FROM pesometros WHERE fecha between '2019-08-07 08:00:03' and '2019-08-16 22:47:00' 
[254724709600, 265377798100]
[84099887700, 87408469800]
0  2019-08-07 19:20:13   254724709600   0---------84099887700   0
1  2019-08-07 19:22:44   254724709600   0---------84099887700   0
2  2019-08-07 19:23:37   254724709600   0---------84099887700   0
3  2019-08-07 19:27:11   254724709600   0---------84099887700   0
4  2019-08-07 19:40:21   254724709600   0---------84099887700   0
5  2019-08-07 19:40:23   254724709600   0---------84099887700   0
6  2019-08-07 19:40:26   254724709600   0---------84099887700   0
7  2019-08-07 19:40:29   254724709600   0---------84099887700   0
8  2019-08-07 19:40:32   254724709600   0---------84099887700   0
9  2019-08-07 19:40:36   254724709600   0---------84099887700   0
10  2019-08-07 19:40:39   254724709600   0---------84099887700   0
11  2019-08-07 19:40:42   254724709600   0---------84099887700   0
12  2019-08-07 